# Game Simulation Pipeline

This notebook walks through the process of building a basketball game simulator with the objective of  The code here is optimized for a Monte Carlo simulation with modularized functions for quick replacement of deterministic methods with stochastic methods. 

__________

In [14]:
from dataclasses import dataclass, field
from typing import List, Dict
import random

#### Object Oriented Approach

Optimizing data organization is critical for efficient handling of Monte Carlo iterations and stochastic modeling. We want to balance structure with memory and computational efficiency. I have decided to take an object oriented approach, breaking down the simulation task into the following classes:

- **Simulation**: Handles the iterations of the Monte Carlo simulation along with aggregating the results from each game simulation.
- **Game**: Handles the game context for a single game simulation - clock, posession, team fouls, timeouts, etc.
- **PlayerGameState**: Handles the *mutable* in-game stats for a player - points, assists, rebounds, personal fouls, etc.
- **PlayerProfile**: Handles the *immutable* historical data retrieved from a database at the start of the simulation.

The separation of the mutable data from the immutable data is crutial for minimizing the memory required to run thousands of simulations of the same game. Here, a player profile will be generated once at the start of the simulation. Empty PlayerGameState objects will be initilized for tracking in-game stats and reference the same player profile objects afor any read-only data required from the player. This approach allows the Player profiles to be established only once, instead of requiring deep copies or clones to create new player profiles for each simulation.

### Simulating a Possession

A baseketball posession can be broken into a series of decisions that make up a decision tree. Some of these decisions are offensively oriented (shot taken), defensive oriented (foul before shot), or require interactions between both (rebounds).

We start each possession under the assumption that two things can happen: Offense attempts to run a play or Defense commits a foul to prevent that play from executing. We categorize this differently from a foul that occurs during the play execution (shooting foul).

In [ ]:
@dataclass
class LivePlayerState:
    points: int = 0
    assists: int = 0
    rebounds: int = 0
    turn_overs: int = 0
    minutes: float = 0.0
    shots: int = 0
    two_pt_shots_taken: int = 0
    two_pt_shots_made: int = 0
    three_pt_shots_taken: int = 0
    three_pt_shots_made: int = 0
    fg_percent: float = 0.0

class PlayerProfile:
    def __init__(self, params):
        self.name: str = params['name']
        self.number: int = params['j_number']
        self.team: str = params['team']
        self.historical_stats = params['historical_data']
        self.fg_pct: float = params['fg_pct']
        self.two_pt_pct: float = params['two_pt_pct']
        self.three_pt_pct: float = params['three_pt_pct']
        self.ft_pct: float = params['ft_pct']
        self.pts_pg: float = params['pts_pg']
        self.ast_pg: float = params['ast_pg']
        self.reb_pg: float = params['reb_pg']
        self.min_pg: float = params['min_pg']
        self.shot_pg: float = params['shot_pg']
        self.to_pg: float = params['to_pg']

    def __str__(self):
        print(f"Player Profile Object: {self.name}")

class PlayerGameState:
    def __init__(self, profile):
        self.name: str = ""
        self.team: str = ""
        self.historical_stats: PlayerProfile = profile
        self.live_stats: LivePlayerState = LivePlayerState()

    def shot_taken(self, shot_type):
        valid_shots = ('1','2','3')
        if shot_type not in valid_shots:
            raise ValueError(f"Invalid shot type: '{shot_type}'. Expected '1','2', or '3'.")

        if shot_type == '2':
            return random.binomial(1, self.historical_stats.two_pt_pct)
        elif shot_type == '3':
            return random.binomial(1, self.historical_stats.three_pt_pct)
        else: # shot_type == 1 Free Throw
            return random.binomial(1, self.historical_stats.ft_pct)


In [ ]:
@dataclass
class LiveTeamState:
    name: str = ""
    players: List[PlayerGameState] = field(default_factory=list)
    start_lineup: List[PlayerGameState] = field(default_factory=list)
    live_lineup: List[PlayerGameState] = field(default_factory=list)
    points: int = 0
    team_fouls: int = 0
    timeouts: int = 5
    agg_foul_prob: float = 0.0
    agg_turnover_prob: float = 0.0
    norm_foul_prob: List[float] = field(default_factory=list)
    norm_turnover_prob: List[float] = field(default_factory=list)

class Game:
    def __init__(self, HomeTeam:str, AwayTeam:str):
        self.clock = 12*60
        self.quarter = 1
        self.HT = LiveTeamState(name = HomeTeam)
        self.AT = LiveTeamState(name = AwayTeam)
        self.OFF = None
        self.DEF = None
        self.possession_arrow = None

    def __str__(self):
        print(f"{self.AT} vs. {self.HT}")

    def simulate(self):
        self.start_game()
        while (self.clock > 0) & (self.quarter <= 4):
            posession_data = self.simulate_posession()
            self.update_game(posession_data)
    
    def start_game(self):
        '''
        This function performs the following tasks:
        1. Determines first possession (tipoff) and posession arrow
        2. Establishes starting lineup
        3. Establishes the player stat distribution that the Monte Carlo simulation will sample from
        '''
        self.rotate_lineup()
        if self.tipoff():
            self.OFF = self.HT
            self.DEF, self.possession_arrow = self.AT
        else:
            self.OFF = self.AT
            self.DEF, self.possession_arrow = self.HT
        
        pass

    def player_distributions(self):
        '''
        This function establishes player stat distributions 
        '''
    
    def simulate_possession(self):
        self.non_shooting_foul()

        self.offensive_play()
            
        
    def non_shooting_foul(self):
        ''' 
        This function will most likely use an XGBoost model or a probability distribution to determine 
        whether a non-shooting foul interrupted the possession before a shot was taken or a turnover committed.
        For now it is represented as a static probability.'''

        if bool(random.binomial(1, self.DEF.foul_prob)):
            # determine who committed the foul

    def shooting_foul(self):
        '''
        This function determines whether a shooting foul was committed and if so, who committed it
        '''
        
    def offensive_play(self):
        '''
        This function determines whether an offensive play resulted in a turnover or a shot taken. 
        '''
        

    def rotate_lineup(self, team):
        '''
        This function will require recalculating probabilities depending on which lineup is on the floor.
        For example, when players are subbed into the game, there is a new likelihood of a foul and turnover being committed based on the lineup.
        These new probabilities will be updated as game state parameters'''

        valid_teams = ('HT', 'AT')
        if team not in valid_teams:
            raise ValueError(f"Invalid team passed to the lineup rotation: {team}. Expected 'HT' or 'AT'.")

        if team == 'HT':
            team = self.HT
        else:
            team == self.AT

        team.agg_foul_prob = sum(player['foul_prob'] for player in team.live_lineup)
        team.norm_foul_prob = [player['foul_prob'] / team.agg_foul_prob for player in team.live_lineup]


    def update_game(self):
        '''
        Update: Possesssion, 
        '''
        pass

    def tipoff(self):
        ''' 50/50 tipoff for now'''
        return bool(random.binomial(1,0.5))

    def reset_game(self):
        pass

In [ ]:
class Simulation:
    def __init__(self, n_games:int, HomeTeam:str, AwayTeam:str):
        self.monte_iter = n_games
        self.home_team = HomeTeam
        self.away_team = AwayTeam
        self.game = Game(HomeTeam=self.home_team, AwayTeam=self.away_team)
        self.results = []
    
    def run_simulation(self):
        self.simulation_setup()
        for _ in range(self.monte_iter):
            self.game.simulate()
            self.results.append(self.game.get_results()) # Probably a better way to store the results of each monte carlo iteration to leverage matrix operations
            self.game.reset()
    def simulation_setup(self):
        ''' 
        When a simulatihon is started, all the player objects are initialized and the game object is initialized.
        1. Use the team names to query the database and get historical data on the players from both teams.
        
        '''
        
        

